In [185]:
import psycopg2


Addresses of local dbs

In [186]:
def jaro_winkler(string1, string2):
    """
    Custom implementation of the Jaro-Winkler similarity.
    :param string1: First string.
    :param string2: Second string.
    :return: Jaro-Winkler similarity score as a float.
    """
    def get_matching_characters(s1, s2, max_distance):
        """Find characters that match within max_distance."""
        matched = []
        matched_indices = set()
        for i, char1 in enumerate(s1):
            for j in range(max(0, i - max_distance), min(len(s2), i + max_distance + 1)):
                if char1 == s2[j] and j not in matched_indices:
                    matched.append(char1)
                    matched_indices.add(j)
                    break
        return matched

    def get_transpositions(s1_matches, s2_matches):
        """Count the number of transpositions between matched characters."""
        return sum(1 for c1, c2 in zip(s1_matches, s2_matches) if c1 != c2) // 2

    # Step 1: Jaro similarity
    max_distance = (max(len(string1), len(string2)) // 2) - 1
    string1_matches = get_matching_characters(string1, string2, max_distance)
    string2_matches = get_matching_characters(string2, string1, max_distance)

    if not string1_matches or not string2_matches:
        return 0.0

    m = len(string1_matches)
    t = get_transpositions(string1_matches, string2_matches)
    jaro = (m / len(string1) + m / len(string2) + (m - t) / m) / 3

    # Step 2: Jaro-Winkler adjustment
    prefix = 0
    for c1, c2 in zip(string1, string2):
        if c1 == c2:
            prefix += 1
        else:
            break
        if prefix == 4:  # Only consider up to 4 characters for prefix scaling
            break

    scaling_factor = 0.1  # Winkler's scaling factor
    jaro_winkler = jaro + prefix * scaling_factor * (1 - jaro)
    return jaro_winkler

# Example usage
score = jaro_winkler("martha", "marhta")
print(f"Jaro-Winkler Score: {score}")


Jaro-Winkler Score: 0.9611111111111111


'air india': {'Flight code': 'flightcode',
  'Status': 'currentstatus',
  'Departure_Time': 'departuretime',
  'Landing_Time': 'arrivaltime',
  'Source/From': 'departureairport',
  'Destination': 'destinationairport',
  'Distance': 'flightdistance',
  'Delay': 'averagedelay',
  'Economy Seats': 'economy_seats',
  'Business_Seats': 'business_seats',
  'First Class-Seats': 'firstclass_seats',
  'Economy Price': 'ticketprice_economy',
  'Business Price': 'ticketprice_business',
  'First Class Price': 'ticketprice_firstclass'},
 'indigo': {'Flight code': 'flight_code',
  'Status': 'status',
  'Departure_Time': 'starttime',
  'Landing_Time': 'endtime',
  'Source/From': 'fromlocation',
  'Destination': 'tolocation',
  'Distance': 'distancekm',
  'Delay': 'avgdelay',
  'Business_Seats indigo':'',
  'Business Price indigo':'',
  'Economy Price': 'flightfare',
  'Economy Seats': 'seatsleft',
  'First Class-Seats': '',
  'First Class Price': ''},
 'vistara': {'Flight code': 'flightid',
  'Status': 'flightstatus',
  'Departure_Time': 'takeofftime',
  'Landing_Time': 'landtime',
  'Source/From': 'origin',
  'Destination': 'destination',
  'Distance': 'distancetraveled',
  'Delay': 'estimateddelay',
  'Economy Seats': 'availableseats_economy',
  'Business_Seats': 'availableseats_business',
  'First Class-Seats': '',
  'Economy Price': 'priceperseat_economy',
  'Business Price': 'priceperseat_business',
  'First Class Price': ''}}

In [212]:
dbnames={"air india","indigo","vistara"}

In [213]:
field_index={}
for db in dbnames:
    field_index[db]={}

In [279]:
dpix={"yi":90}
type(list(dpix.items())[0])

tuple

In [ ]:
def custom_query(connection_params,query):
    try:
        # Establish connection
        conn = psycopg2.connect(**connection_params)
        print("Connection successful")

        # Create a cursor object
        cursor = conn.cursor()

        # Execute a query
        cursor.execute(f"{query}")

        # Fetch and print the result
        results= cursor.fetchall()
        # print(f"PostgreSQL version: {version}")
        column_names = [desc[0] for desc in cursor.description]
        


        # Close cursor and connection
        cursor.close()
        conn.close()
        # print(column_names)
        for i in range(len(column_names)):
            
            field_index[connection_params['dbname']][column_names[i]]=i
        return results
    except Exception as e:
        print(f"Error: {e}")
        return None

    

In [215]:
Connection_params={
    "air india":{
    "dbname": "air india",
    "user": "postgres",
    "password": "addy",
    "host": "localhost",  # Use the actual host (e.g., "127.0.0.1" or server IP)
    "port": 5432          # Default PostgreSQL port
    },
    "vistara":{
    "dbname": "vistara",
    "user": "postgres",
    "password": "addy",
    "host": "localhost",  # Use the actual host (e.g., "127.0.0.1" or server IP)
    "port": 5432          # Default PostgreSQL port
    },
    "indigo":{
    "dbname": "indigo",
    "user": "postgres",
    "password": "addy",
    "host": "localhost",  # Use the actual host (e.g., "127.0.0.1" or server IP)
    "port": 5432          # Default PostgreSQL port
    }
}

In [216]:
# query is select * from (all combined customer tables tables)
fkey_mapping={
    "air india":"",
    "indigo":"emailid",
    "vistara":""
}

In [217]:
combined_table_mapping={}

In [218]:
#dictionary for mapping 
table_mapping={
    "air india":["customer_info"],
    "indigo":["traveller","loyalty"],
    "vistara":["passenger"]
    
}

In [219]:
for k,v in table_mapping.items():
    joined_table=v[0]
    for i in range(1,len(v)):
        
        joined_table+=f" join {v[i]} on {v[i-1]}.{fkey_mapping[k]} = {v[i]}.{fkey_mapping[k]}"
    combined_table_mapping[k]=joined_table



In [220]:
combined_table_mapping

{'air india': 'customer_info',
 'indigo': 'traveller join loyalty on traveller.emailid = loyalty.emailid',
 'vistara': 'passenger'}

In [221]:
#dictionary for all customer data:
cust_data={}
for k,v in Connection_params.items():
    query=f"select * from {combined_table_mapping[k]} ;"
    cust_data[k]=custom_query(v,query)
    
    

Connection successful
['CustomerID', 'FullName', 'EmailAddress', 'ContactNumber', 'LoyaltyPoints', 'dataofbirth', 'cityofresidence', 'address']
Connection successful
['passengerid', 'passengername', 'email', 'contact', 'loyaltypoints', 'dataofbirth', 'city', 'address']
Connection successful
['customerid', 'full_name', 'emailid', 'contactno', 'loyaltypoints', 'dob', 'city', 'address', 'emailid', 'points']


In [ ]:
cust_data['air india']

global schema for customer-
Customer = [
    "Name", "Email", "Phone no.", "Point/frequent_flyer_miles", 
    "Age Group", "Medical Condition (optional)", 
    "City", "Address"
]

In [239]:
# mapping from schema matching
schema_mapping={
    "air india":{
    
    "Name":"FullName", "Email":"EmailAddress", "Phone no.":"ContactNumber", "Points":"LoyaltyPoints", 
    "Date of Birth":"dataofbirth", "City":"cityofresidence","address":"address"
        
    },
    # "Indigo":{
    #     "Name":"full_name", "Email":"emailid", "Phone no.":"contact", "Point/frequent_flyer_miles":"points", 
    # "Date of Birth": "dob", 
    # "City":"city"
    # },
     "indigo":{
    
    "Name":"full_name", "Email":"emailid", "Phone no.":"contactno", "Points":"loyaltypoints", 
    "Date of Birth":"dob", "City":"city","address":"address"
        
    },

    # "Vistara":{
    #       "Name":"passenger_name", "Email":"email", "Phone no.":"contactno", "Point/frequent_flyer_miles":"frequentflyerpoints", 
    # "Age": "dateofbirth", 
    # "City":"city"
    # }
     "vistara":{
    
    "Name":"passengername", "Email":"email", "Phone no.":"contact", "Points":"loyaltypoints", 
    "Date of Birth":"dataofbirth", "City":"city","address":"address"
        
    }


}

Blocking based on city

In [ ]:
import pandas as pd

In [230]:
dbnames=["air india","indigo","vistara"]

In [268]:
# del cust_data['Indigo']
city_blocks={}
for db in dbnames:
    city_blocks[db]={}

for airline, data in cust_data.items():
    for row in data:
        if row[field_index[airline][schema_mapping[airline]["City"]]] not in city_blocks[airline]:
            city_blocks[airline][row[field_index[airline][schema_mapping[airline]["City"]]]]=[row]
        else:
            city_blocks[airline][row[field_index[airline][schema_mapping[airline]["City"]]]].append(row)


blocks=city_blocks

for airline, cities in city_blocks.items():
    for city,rows in cities.items():
        blocks[airline][city]={}
        for row in rows:
            if row[field_index[airline][schema_mapping[airline]["Date of Birth"]]] not in blocks[airline][city]:
                blocks[airline][city][row[field_index[airline][schema_mapping[airline]["Date of Birth"]]]]=[row]
            else:
                blocks[airline][city][row[field_index[airline][schema_mapping[airline]["Date of Birth"]]]].append(row)

            






#Matching

In [269]:
import regex as re
def preprocess_address(address):
    """
    Preprocess an address string: convert to lowercase and split into words.

    Args:
        address (str): The address string.

    Returns:
        set: A set of normalized words in the address.
    """
    # Convert to lowercase
    address = address.lower()
    # Remove punctuation
    address = re.sub(r'[^\w\s]', '', address)
    # Split into words and convert to a set
    return set(address.split())

def jaccard_similarity_addresses(addr1, addr2):
    """
    Calculate the Jaccard similarity between two addresses.

    Args:
        addr1 (str): The first address.
        addr2 (str): The second address.

    Returns:
        float: Jaccard similarity score between 0 and 1.
    """
    set1 = preprocess_address(addr1)
    set2 = preprocess_address(addr2)
    
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0.0

In [270]:
# "Name":"fullname", "Email":"emailaddress", "Phone no.":"contactnumber", "Point/frequent_flyer_miles":"loyaltypoints", 
#     "Date of Birth":"Date of Birth", "City":"cityofresidence","address":"address"
        
def calculate_sim(row1,row2,airline1,airline2):
    email1=row1[field_index[airline1][schema_mapping[airline1]['Email']]]
    email2=row2[field_index[airline2][schema_mapping[airline2]['Email']]]

    phone1=row1[field_index[airline1][schema_mapping[airline1]['Phone no.']]]
    phone2=row2[field_index[airline2][schema_mapping[airline2]['Phone no.']]]
    
    if email1==email2 or phone1==phone2:
        return True

    address1=row1[field_index[airline1][schema_mapping[airline1]['address']]]
    address2=row1[field_index[airline2][schema_mapping[airline2]['address']]]

    name1=row1[field_index[airline1][schema_mapping[airline1]['Name']]]
    name2=row2[field_index[airline2][schema_mapping[airline2]['Name']]]

    name_score=jaro_winkler(name1,name2)
    address_score=jaccard_similarity_addresses(address1,address2)
    
    
    if(name_score+address_score>1.7):
        return True
    
    return False
    
    

In [272]:
df = pd.DataFrame(columns=["Name", "Date of Birth", "City","email-air india","email-indigo","email-vistara","points-air india","points-indigo","points-vistara","address","Phone no."])

Structure of blocks---
Dictionary-

In [273]:
for i in range(len(dbnames)):
    for city,dobs in blocks[dbnames[i]].items():
        for dob,rows in blocks[dbnames[i]][city].items():
            for row in blocks[dbnames[i]][city][dob]:
                df_row={"Name":"", "Date of Birth":"", "City":"","email-air india":"","email-indigo":"","email-vistara":"","points-air india":0,"points-indigo":0,"points-vistara":0,"address":"","Phone no.":""}
                for key in df_row.keys():
                    if key in schema_mapping[dbnames[i]]:
                        
                        df_row[key]=row[field_index[dbnames[i]][schema_mapping[dbnames[i]][key]]]
                
                df_row[f"email-{dbnames[i]}"]=row[field_index[dbnames[i]][schema_mapping[dbnames[i]]["Email"]]]
                df_row[f"points-{dbnames[i]}"]=row[field_index[dbnames[i]][schema_mapping[dbnames[i]]["Points"]]]

                for j in range(i+1,len(dbnames)):
                    row1=row
                    if city in blocks[dbnames[j]] and dob in blocks[dbnames[j]][city]:

                        # for z,row2 in enumerate(blocks[dbnames[j]][city][dob]):
                        z=0
                        while z<len(blocks[dbnames[j]][city][dob]):
                            row2=blocks[dbnames[j]][city][dob][z]
                            airline1=dbnames[i]
                            airline2=dbnames[j]
                            if calculate_sim(row1,row2,airline1,airline2):
                                df_row[f"email-{dbnames[j]}"]=row[field_index[dbnames[j]][schema_mapping[dbnames[j]]["Email"]]]
                                df_row[f"points-{dbnames[j]}"]=row[field_index[dbnames[j]][schema_mapping[dbnames[j]]["Points"]]]
                                del blocks[dbnames[j]][city][dob][z]
                                break
                            else:
                                z+=1
                new_row_df = pd.DataFrame([df_row])
                df = pd.concat([df, new_row_df], ignore_index=True)


                    
                
            
        
            


In [ ]:
def get_all(dbname):
    count=0
    for city,dobs in blocks[dbname].items():
        for dob, rows in dobs.items():
            count+=len(rows)
    
    return count
            

In [ ]:
get_all('Vistara')

In [ ]:
blocks['Indigo']['Hyderabad']["1963-01-05"]

In [274]:
df.shape

(178, 11)

In [275]:
df.to_csv("combined.csv",index=False)
